# Dummy Column

Using dask futures API to pass cuDF column and Series objects between workers. Columns work but Series do not.

In [1]:
from dummy import DummyColumn

In [2]:
type(DummyColumn())

dummy.DummyColumn

In [3]:
d = DummyColumn()
h, f = d.serialize()
print(d.deserialize(h, f), f, type(f[0]))

[1 2 3] [<numba.cuda.cudadrv.devicearray.DeviceNDArray object at 0x7f0182874ba8>] <class 'numba.cuda.cudadrv.devicearray.DeviceNDArray'>


In [4]:
import os

In [5]:
base_env = {
    "NOTEBOOK": "1",
    "UCX_RNDV_SCHEME": "put_zcopy",
    "UCX_MEMTYPE_CACHE": "n",
    "UCX_TLS": "rc,cuda_copy,cuda_ipc",
    "CUDA_VISIBLE_DEVICES": "4,5",
}
os.environ.update(base_env)

In [6]:
# from numba import cuda
# from distributed.protocol.cuda import cuda_serialize, cuda_deserialize

# class DummyColumn(object):                                                                                                                                                                                      
                                                                                                                                                                                                                
#     def __init__(self):                                                                                                                                                                                         
#         import threading                                                                                                                                                                                        
#         self.lock = threading.Lock()                                                                                                                                                                            
#         pass                                                                                                                                                                                                    
                                                                                                                                                                                                                
#     def serialize(self):                                                                                                                                                                                        
#         header = {}                                                                                                                                                                                             
#         frames = [cuda.to_device(np.arange(100000))]                                                                                                                                                            
#         return header, frames                                                                                                                                                                                   
                                                                                                                                                                                                                
#     @classmethod                                                                                                                                                                                                
#     def deserialize(cls, header, frames):                                                                                                                                                                       
#         return np.array([1, 2, 3])                                                                                                                                                                              
                                                                                                                                                                                                                
# @cuda_serialize.register(DummyColumn)                                                                                                                                                                           
# def serialize(col):                                                                                                                                                                                             
#     return col.serialize()                                                                                                                                                                                      
                                                                                                                                                                                                                
# @cuda_deserialize.register(__main__.DummyColumn)                                                                                                                                                                         
# def deserialize(header, frames):                                                                                                                                                                                
#     return DummyColumn.deserialize(header, frames) 

In [7]:
# from cudf.core.column.column import DummyColumn

In [8]:
import dask
import distributed
import pandas as pd
import os
import numpy as np

In [9]:
print(f'Dask: {dask.__file__}')
print(f'Distributed: {distributed.__file__}')

Dask: /home/nfs/bzaitlen/GitRepos/dask/dask/__init__.py
Distributed: /home/nfs/bzaitlen/GitRepos/distributed/distributed/__init__.py


In [10]:
from dask.distributed import Client, wait
from dask_cuda import DGX

#cluster = DGX(CUDA_VISIBLE_DEVICES=[2,3], 
#              dashboard_address='10.33.227.165:8789')
#client = Client(cluster)
client = Client('ucx://10.33.225.165:8986')
client

Client Scheduler: ucx://10.33.225.165:8986 Dashboard: http://10.33.225.165:8787/status,Cluster Workers: 2 Cores: 2 Memory: 64.00 GB


In [11]:
import sys, os

In [12]:
path = os.getcwd()

In [13]:
def set_os(path):
    import sys
    sys.path.append(path)
    return sys.path

In [14]:
client.run(set_os, path)

{'ucx://10.33.225.185:34114': ['/home/nfs/bzaitlen/worker-4sawvncc',
  '/home/nfs/bzaitlen/miniconda3/envs/cudf-dev/bin',
  '/home/nfs/bzaitlen/miniconda3/envs/cudf-dev/lib/python37.zip',
  '/home/nfs/bzaitlen/miniconda3/envs/cudf-dev/lib/python3.7',
  '/home/nfs/bzaitlen/miniconda3/envs/cudf-dev/lib/python3.7/lib-dynload',
  '/home/nfs/bzaitlen/.local/lib/python3.7/site-packages',
  '/home/nfs/bzaitlen/miniconda3/envs/cudf-dev/lib/python3.7/site-packages',
  '/home/nfs/bzaitlen/miniconda3/envs/cudf-dev/lib/python3.7/site-packages/nvstrings_cudaunknown-0.0.0.dev0-py3.7-linux-x86_64.egg',
  '/home/nfs/bzaitlen/GitRepos/cudf/python/cudf',
  '/home/nfs/bzaitlen/GitRepos/cudf/python/dask_cudf',
  '/home/nfs/bzaitlen/GitRepos/distributed',
  '/home/nfs/bzaitlen/GitRepos/dask',
  '/home/nfs/bzaitlen/GitRepos/dask-ml',
  '/home/nfs/bzaitlen/GitRepos/dask-cuda',
  '/home/nfs/bzaitlen/GitRepos/ucx-py',
  '/home/nfs/bzaitlen/GitRepos/pynvml',
  '/home/nfs/bzaitlen/GitRepos/cudf-ucx-fail-scripts'

In [15]:
worker_1, worker_2 = client.scheduler_info()['workers']

In [ ]:
from dask.distributed import wait
import operator

left = client.map(lambda x: DummyColumn(), range(10000), workers=[worker_1])
right = client.map(lambda x: DummyColumn(), range(10000), workers=[worker_2])
results = client.map(lambda x,y: (x,y), left, right)
_ = wait(results)

In [ ]:
results[0].result()